# 模块A：高性能网络引擎（Network Engine）测试

覆盖：I/O 复用 / EventLoop、连接生命周期、TCP/UDP、连接数限制、内存分配与统计、多线程。

提示：默认使用 `../build` 目录；如需重新编译，把 `BUILD_FIRST = True`。

In [ ]:
from __future__ import annotations

import os
import subprocess
from pathlib import Path

ROOT = (Path("..") if Path.cwd().name == "notebooks" else Path(".")).resolve()
BUILD_DIR = (ROOT / "build").resolve()

TIMEOUT_S = 30
BUILD_FIRST = False
CONTINUE_ON_FAIL = False

TESTS = [
    # I/O + event loop
    "test_eventloop",
    "test_select_poller",
    # TCP/UDP + connection lifecycle
    "test_tcpserver",
    "test_tcpclient",
    "test_connector_retry",
    "test_idle_cleanup",
    "test_udp_proxy",
    "test_l4_tunnel",
    # connection limits
    "test_conn_limit",
    "test_conn_limit_user_service",
    # memory/allocator
    "test_allocator",
    "test_buddy_allocator",
    "test_memory_stats",
    "test_hugepage_slab",
    # threading model
    "test_multithread",
]


In [ ]:
def sh(cmd: list[str], *, cwd: Path = ROOT, timeout: int | None = None) -> None:
    print("$", " ".join(cmd))
    subprocess.run(cmd, cwd=str(cwd), check=True, timeout=timeout)


def maybe_build() -> None:
    BUILD_DIR.mkdir(parents=True, exist_ok=True)
    cache = BUILD_DIR / "CMakeCache.txt"
    if not cache.exists():
        sh(["cmake", "-S", str(ROOT), "-B", str(BUILD_DIR)])
    jobs = str(max(1, os.cpu_count() or 4))
    sh(["cmake", "--build", str(BUILD_DIR), "-j", jobs])


def run_tests() -> None:
    failures: list[tuple[str, str]] = []
    for t in TESTS:
        p = BUILD_DIR / t
        if not p.exists():
            print(f"SKIP {t} (missing: {p})")
            continue
        print(f"RUN {t}")
        try:
            sh([str(p)], cwd=ROOT, timeout=TIMEOUT_S)
        except subprocess.TimeoutExpired:
            print(f"FAIL {t} timeout={TIMEOUT_S}s")
            failures.append((t, "timeout"))
            if not CONTINUE_ON_FAIL:
                break
        except subprocess.CalledProcessError as e:
            print(f"FAIL {t} exit={e.returncode}")
            failures.append((t, f"exit={e.returncode}"))
            if not CONTINUE_ON_FAIL:
                break

    if failures:
        raise RuntimeError("Failed tests: " + ", ".join([f"{n}({r})" for n, r in failures]))
    print("OK")


if BUILD_FIRST:
    maybe_build()
run_tests()
